# Deteksi Wajah dan Senyum dengan OpenCV

Notebook ini mencakup tahapan:
1. Persiapan dan Konfigurasi
2. Konfigurasi Dataset (real-time dari webcam)
3. Eksplorasi Data
4. Pre-processing Gambar


In [ ]:

import cv2
import os

# Load Haar Cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
smile_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_smile.xml')

# Cek jika berhasil dimuat
assert not face_cascade.empty(), "Gagal memuat model deteksi wajah."
assert not smile_cascade.empty(), "Gagal memuat model deteksi senyum."


## Konfigurasi Dataset
Dataset yang digunakan bersifat real-time, diambil dari webcam pengguna. Tidak diperlukan file dataset eksternal.


In [ ]:

# Buka webcam
cap = cv2.VideoCapture(0)

# Folder penyimpanan hasil deteksi senyum
save_folder = "hasil_smile_scan"
os.makedirs(save_folder, exist_ok=True)


## Eksplorasi Data
Kita akan menghitung jumlah wajah dan senyum yang terdeteksi di setiap frame webcam.


In [ ]:

saved = False  # Simpan hanya satu kali

while True:
    ret, frame = cap.read()
    if not ret:
        break

    original_frame = frame.copy()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Pre-processing

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    wajah_terdeteksi = len(faces)
    senyum_terdeteksi = 0

    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        smiles = smile_cascade.detectMultiScale(roi_gray, scaleFactor=1.3, minNeighbors=10, minSize=(20, 20))

        if len(smiles) > 0 and not saved:
            filename = os.path.join(save_folder, "senyum_terdeteksi.jpg")
            cv2.imwrite(filename, original_frame)
            saved = True

        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        for (sx, sy, sw, sh) in smiles:
            cv2.rectangle(frame, (x + sx, y + sy), (x + sx + sw, y + sy + sh), (0, 255, 0), 2)

    cv2.putText(frame, f'Wajah: {wajah_terdeteksi}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)
    cv2.putText(frame, f'Senyum: {senyum_terdeteksi}', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
    cv2.imshow("Deteksi Wajah dan Senyum", frame)

    if cv2.waitKey(1) & 0xFF == 32:
        break

cap.release()
cv2.destroyAllWindows()
